<div style="font-family: 'Times New Roman'; font-size: 16px;">

# Quantum Linear Algebra
The Harrow–Hassidim–Lloyd (HHL) algorithm [Project 1]  
*By Sonny Lowe, David Lee, Arav Raval*

</div>


<div style="font-family: 'Times New Roman'; font-size: 16px;">

This notebook will discuss the Harrow-Hassidim-Lloyd (HHL) quantum algorithm, meant for solving a linear system
$$A\vec{x}=\vec{b} \text{ ,  where } A \text{ is a hermitian matrix}$$
and where $\vec{x}$ and $\vec{b}$ ultimately represent quantum states $\ket{x}$ and $\ket{b}$ respectively. We will provide a derivation, implementation, generalization to non-hermitian matrices, as well as the context for HHL as a subroutine.

Note: we will generally be working in the normalized domain.
</div>

<div style="font-family: 'Times New Roman'; font-size: 16px;">

### Section 1: Mathematical Derivation
 
Our problem is represented as 
$$A\ket{x} = \ket{b}$$
where $\ket{b} \in \mathbb{C}^N$ is some given quantum state and $A \in \mathbb{C}^{N\times N}$. Our goal is to solve for $\ket{x} \in \mathbb{C}^N$ under a few conditions:
- $A$ is hermitian such that $A = A^\dagger$
- $A$ is $s$-sparse and well conditioned, meaning it has at most $s$ nonzero entries per row and its condition number  $\kappa(A)$ is relatively small such that the system is stable and less sensitive to perturbations.
We will break down the derivation into several steps.
- We have access to an "oracle" of A in that we have its eigenvalues to use in our circuit (potential weakness).
- We assume there exists some efficient algorithm to prepare $\ket{b}$

<br>

#### **1. Rewriting and Initialization**
Given that $A$ is a hermitian matrix, there exists a spectral decomposition such that the matrix can be diagonalized by unitary transformations.
$$A = UDU^T,\quad \text{ where } U \text{ is a unitary matrix and } D \text{ is diagonal composed of the real eigenvalues of } A$$
Since the columns of $U$ form an orthonormal basis and are the eigenvalues, we can rewrite this decomposition where $|u_{i}\rangle$ is the $i^{th}$ eigenvector of $A$ with respective eigenvalue $\lambda_{i}$ as:
$$A = \sum_{i=0}^{N-1}\lambda_{i}\ket{u_{i}}\bra{u_{i}}, \quad \lambda_{i}\in\mathbb{ R }$$
Likewise, as a linear transformation, we can write our resultant vector $\ket{b}$ in the eigenbasis of $A$.
$$\ket{b} = \sum_{i=0}^{N-1}b_{i}\ket{u_{i}}, \quad b_{i}\in\mathbb{ C }$$
Thus, our problem can now be rewritten as:
$$\ket{x} = A^{-1}\ket{b} = \sum_{i=0}^{N-1}\frac{1}{\lambda_{i}}b_{i}\ket{u_{i}}\bra{u_{i}}u_{i}\rangle = \sum_{i=0}^{N-1}\frac{1}{\lambda_{i}}b_{i}\ket{u_{i}}$$

Moreover, note that to represent a state vector $\ket{b}$ of dimension $N$, to prepare it as a quantum register, we prepare it in binary basis using $\log_2{N}$ qubits. Therefore, from here on, $N = \log_2{N}$.

<br>

#### **2. Quantum Phase Estimation**
The problem first boils down to being able to decompose a matrix to find its eigenvalues and eigenvectors in a computationally efficient manner. At a high level, Quantum Phase Estimation is a procedure that performs a series of controlled-$U$ gates given some unitary matrix $U$ with eigenvalues of the form $e^{2\pi i \theta}$, and finds $\theta$.

For HHL, first, we start with two registers $\ket{0}^{\otimes N}$ and $\ket{b}$. Our initial state is thus $\ket{\Phi_0} = \ket{0}^{\otimes N} \otimes \ket{b}.$ Then we can characterize the behavior of **QPE** as:
$$\textbf{QPE}(U,\ket{0}^{\otimes N},\ket{b}) = \textbf{QPE}(U,\ket{0}^{\otimes N},\sum_{j=0}^{N-1}b_{j}\ket{u_{j}}) = \sum_{j=0}^{N-1}\ket{\lambda_{j}}b_{j}\ket{u_{j}}, \quad\text{ for some clever choice of } U$$

We will choose $U = e^{iAt}$ for some constant $t$ such that $U$ is governed by the same eigenvalues of $A$ and we have that, where $|u_{i}\rangle$ is the $i^{th}$ eigenvector of $A$ with respective eigenvalue $\lambda_{i}$:
$$U = e^{iAt} = \sum_{j=0}^{N-1}e^{i\lambda_{j}t}\ket{u_j}\bra{u_j}$$

>1. First, we hadamard our first register of $\ket{0}^{\otimes N}$. $H^{\otimes N}\ket{\Phi_0} = \frac{1}{2^{n/2}}(\ket{0} + \ket{1})^{\otimes N} \otimes \ket{b}$
>2. Next, we perform a series of $N$ controlled-$U$ gates such that for $k = {0, 1,...,N-1}$ (notice $|k| = N$), we create the gate where $\ket{k}$ is the k-th qubit in the first registrar: 
$$CU^{2^k}(\ket{k}, \ket{b}) =_{\text{if} \ket{k}=\ket{1}} \ket{k} \otimes U^{2^k}\ket{b} = \ket{k} \otimes \sum_{j=0}^{N-1}e^{i\lambda_{j}t2^k}b_{j}\ket{u_{j}}\bra{u_{j}}u_{j}\rangle = \ket{k} \otimes \sum_{j=0}^{N-1}e^{i\lambda_{j}t2^k}b_{j}\ket{u_{j}} $$
The composition of all N of these such gates yields the state (with normalization) where $k = {2^0, 2^1,...,2^{N-1}}$:
$$ \sum_{j=0}^{N-1} \left( \left( \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^{n-1}} e^{i k \lambda_j t} \ket{k} \right) \otimes b_j \ket{u_j} \right)$$
 >3. Now, it is clear that the phase factor actually depends on the control qubits, and so we consider just our first (control) register, which is now superposed through unitary entanglements to the state:
 $$ \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^{n-1}} e^{i k \lambda_j t} \ket{k} $$
 $$ \text{where the phase factor is }\quad \lambda_i t = 2\pi\theta $$
 It is clear now that the encoded phase factor we desire (which represents the eigenvalues of $A$), can be extracted through the inverse quantum fourier transform operation. Thus, applying **QFT$^{-1}$** has the following behavior:
 $$\textbf{QTF}^{-1}(\frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^{n-1}} e^{i k \lambda_j t} \ket{k}) \mapsto \ket{\theta} \quad\text{ which encodes } \theta_j = \lambda_j t / 2\pi = \tilde{\lambda_j}$$
 If we set $t=2\pi l$ for some constant $l$, we have essentially calculated an approximation of our actual eigenvalues after normalization. Thus, for our total state, we are left with:
 $$\sum_{j=0}^{N-1}\ket{\tilde{\lambda_j}}\otimes b_{j}\ket{u_j}$$

<br>

#### **3. Controlled Rotation**
With our new intermediate state $\sum_{j=0}^{N-1}\ket{\tilde{\lambda_{j}}} \otimes b_{j}\ket{u_{j}} = \ket{\tilde{\lambda}} \otimes \ket{b} \approx \ket{\lambda} \otimes \ket{b}$ (working in normalized domain), our next problem is to produce the "inverse eigenvalue." We will do this by applying a series of controlled rotations. These rotations will utilize the control register which now encodes the eigenvalues applied onto a new ancilla register that will extract $\frac{1}{\lambda}$. (Checkpoint: by now, we have created 3 registers with a total state of $\ket{\lambda} \otimes \ket{b} \otimes \ket{0}_{ancilla}$)

To do this, we will encode the ancilla qubits to the following state:
$$\ket{0}_{ancilla} \mapsto \sqrt{1 - \left(\frac{C}{\lambda}\right)^2}\ket{0}+\frac{C}{\lambda}\ket{1}, \quad\text{ where } C < \min_{j}\lambda_j$$
Considering the general state $\ket{\psi} = \cos(\theta/2)\ket{0} + e^{i\phi}\sin(\theta/2)\ket{1}$ allows us to clearly see that $\theta_j = 2\arcsin(C/\lambda_j) \quad \forall j$. We are left with
 $$\sum_{j=0}^{N-1}\ket{\lambda_j}\otimes b_{j}\ket{u_j} \otimes \sqrt{1 - \left(\frac{C}{\lambda}\right)^2}\ket{0}+\frac{C}{\lambda}\ket{1}$$

<br>

#### **4. Uncompute**
Next, we want to uncompute our eigenvalue register by applying the dagger of QPE. This consists of first applying QFT, then simplying applying the conditioned inverse of the unitary matrix $U$ followed by hadamards. We are left with:
 $$\sum_{j=0}^{N-1}\ket{0}\otimes b_{j}\ket{u_j} \otimes \sqrt{1 - \left(\frac{C}{\lambda}\right)^2}\ket{0}+\frac{C}{\lambda}\ket{1}$$

<br>

#### **5. Measure Ancillas**
Finall, we will measure the ancillas, which after normalization, yields the post measurement state if our ancillas have the desired outcome of $\ket{1}$ in the form:
$$\sum_{i=0}^{N-1}\frac{1}{\lambda_{i}}b_{i}\ket{u_{i}}\ket{1}_{ancilla}$$
It is here that we see that this process is not deterministic and can fail up to a probability if our ancilla is in $\ket{0}$. Thus, we will repeat the process of steps 2-4 until success.

<br>

#### **6. Extracting Answer**
We can see now that our registers hold our solution in the form of $A^{-1}\ket{b}$. However, since the state $\ket{x}$ is determined in our registers as a superposition, we cannot measure the full state in one shot without collapsing it to a single basis state. However, by measuring certain observables, you can gain useful information about the state without fully collapsing it. The quantum expectation value $\bra{x}M\ket{x}$ for a given observable $M$ is useful in the context of a subroutine because it can allow us to:
- obtain information about the solution $\ket{x}$ for instance, the probability distribution of components of $\ket{x}$ in a particular basis).
- Measure a function of the solution, such as a dot product or a norm, which might be useful in applications without needing the full solution.

<br>

#### **Our overall circuit diagram:**
<img src="./assets/schematic-1.png" alt="General Design" width="600" />

</div>

<div style="font-family: 'Times New Roman'; font-size: 16px;">

### Section 2: Generalization to Non-Hermitian
The case where $A \in \mathbb{C}^{N\times N}$ is not hermitian is actually quite easy to resolve. Simply construct a new matrix $C \in \mathbb{C}^{2N \times 2N}$:
$$ C = \begin{bmatrix}0&A\\A^{\dagger}&0\end{bmatrix} $$
Notice that $C$ is immediately hermitian:
$$ C^{\dagger} = \begin{bmatrix}0&A^{\dagger}\\A^{\dagger^{\dagger}}&0\end{bmatrix}^T = \begin{bmatrix}0&A\\A^{\dagger}&0\end{bmatrix} $$
and the properties of sparesness and well conditioning remain relative to this new matrix.

Notice now that our problem changes as such:
$$A\vec{x}=\vec{b} \text{ ,  where } A \text{ is non-hermitian} \quad\Rightarrow\quad C\begin{bmatrix}0\\x\end{bmatrix} = \begin{bmatrix}b\\0\end{bmatrix} \text{ ,  where } C \text{ is a hermitian matrix} $$
</div>

<div style="font-family: 'Times New Roman'; font-size: 16px;">

### Section 3: Sample Implementation

<br>

This project utilizes IBM's Qiskit 
- qiskit == 0.44.2
- qiskit-aer == 0.12.2
- qiskit-terra == 0.25.2.1

</div>

In [8]:
# Some generic python and jupyter imports
import numpy as np
from numpy import pi,sqrt

from qiskit import(QuantumCircuit, execute, Aer, ClassicalRegister, QuantumRegister)
backend_svec = Aer.get_backend('statevector_simulator')
backend_qasm = Aer.get_backend('qasm_simulator')

# Visualizations
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt# implementation of a sample case here

In [44]:
###### NOT COMPLETED, MUST MAKE SURE MATRIX IS SPARSE AND WELL DEFINED #####

# Initialization & Oracle Creation
N = 3

# Function to generate a random invertible matrix
def generate_invertible_matrix():
    while True:
        matrix = np.random.rand(N, N) * (np.random.rand(N, N) + 1j)
        if np.linalg.det(matrix) != 0:
            return matrix

A = generate_invertible_matrix()
C = np.block([[np.zeros_like(A), A], [np.conj(A.T), np.zeros_like(A)]])
#print(C)

n_a = 2*N
n_b = 2*N
n = 2*N

def validate_hhlparams(
        A: np.ndarray,
        b: np.ndarray,
):
    if not isinstance(A, np.ndarray) or A.ndim != 2:
        raise TypeError('A must be a 2D numpy matrix')
    elif not isinstance(b, np.ndarray) or b.ndim != 1:
        raise TypeError('b must be a 1D numpy matrix')
    elif A.shape[0] != A.shape[1]:
        raise ValueError("A must be a square matrix.")
    elif A.shape[0] != b.size: 
        raise ValueError("Invalid dimensions for linear system.")
    elif np.linalg.det(A) == 0:
        raise ValueError("A must an invertible matrix.")
    
    # verify matrix sparsity, well conditioned, here
    

In [58]:
def hhl_circuit(
        A: np.ndarray,
        b: np.ndarray,
) -> QuantumCircuit:
    """    
    :param A: A numpy matrix 
    :param b: A numpy vector
    :return: A quantum circuit for the hhl algorithm
    """
    validate_hhlparams(A, b)
    
    # prepare $\ket{b}$ as a quantum register
    nb = int(np.ceil(np.log2(len(b))))
    b_pad = np.pad(b, (0, 2**nb - len(b)))
    b_init_circuit = QuantumCircuit(nb)
    b_init_circuit.initialize(b_pad / np.linalg.norm(b_pad), list(range(nb)))
    
    return b_init_circuit
    
b = np.array([1,2,3], dtype=complex)
circ = hhl_circuit(A, b)
    

┌────────────────────────────────────────┐
q_0: ┤0                                       ├
     │  Initialize(0.26726,0.53452,0.80178,0) │
q_1: ┤1                                       ├
     └────────────────────────────────────────┘

<div style="font-family: 'Times New Roman'; font-size: 16px;">

### Section 4: Subroutines

</div>

In [ ]:
# implementation of subroutines